In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [10]:
df = pd.read_csv('/Volumes/lab-windingm/home/users/cochral/AttractionRig/modelling-behaviour/food-day4-n10/2024-05-28_13-40-52_td7.tracks.csv')

In [12]:
df

,track_id,frame,x_head,y_head,score_head,x_body,y_body,score_body,x_tail,y_tail,score_tail
0,0,0,712.318176,271.768951,1.011335,700.594727,240.347015,1.030125,712.955078,256.155548,0.962833
1,1,0,524.297913,513.093750,1.026701,499.647247,496.322266,0.979338,519.672729,500.964020,0.994172
2,2,0,1008.233276,560.414673,1.008820,1011.749146,527.962891,0.984674,1012.663391,544.820435,0.961453
3,3,0,877.250793,592.816162,0.857007,900.872009,596.430664,0.915430,888.500061,608.436890,1.069667
4,4,0,688.658081,672.812134,1.000886,660.317505,680.722656,0.938659,679.923645,688.246338,0.937761
...,...,...,...,...,...,...,...,...,...,...,...
29522,85,3600,196.137802,661.399963,0.970446,192.429260,697.215698,0.936295,196.016525,680.422119,0.931120
29523,67,3600,1159.960449,759.861572,0.930416,1159.701050,727.873413,0.966980,1159.904663,744.400574,1.017960
29524,70,3600,229.039307,804.929199,0.953161,220.749481,832.636292,1.016043,228.561737,820.359497,0.997118
29525,84,3600,417.188873,1061.156616,0.948094,408.484497,1088.595215,0.962340,412.416504,1076.194824,0.951564


In [17]:

def distance(x, y, x0, y0):
        
    # ask why we dont square root 
    distance_pixels = (x - x0)**2 + (y- y0)**2

    # distance_cm = distance_pixels * pixels_to_cm
    return distance_pixels


distance_accross_frames = []

for track in df['track_id'].unique():

    track_unique = df[df['track_id'] == track]

    # not every track begins at 0; unsure how to handle this when its the centre position we want 

    first_frame = track_unique[track_unique['frame'] == 0]

    x0, y0 = first_frame[['x_body', 'y_body']].values[0]

    for i, row in track_unique.iterrows():
        frame = row['frame']
        x = row['x_body']
        y = row['y_body']

        displacement = distance(x, y, x0, y0) 

        distance_accross_frames.append((frame, displacement))





IndexError: index 0 is out of bounds for axis 0 with size 0